In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

In [ ]:
dataset_path = r'./datasus/part-00000-0b7ee8fc-4d40-4b71-bec3-5d9ddbf54ec9.c000.csv'
datasus_df = pd.read_csv(dataset_path, sep=';')

# Verificando informações gerais sobre o dataset

In [ ]:
datasus_df.info()

In [ ]:
datasus_df.describe()

In [ ]:
datasus_df['classificacaoFinal'].info()

# Verificação de valores nulos

In [ ]:
msno.matrix(datasus_df)

# Conferindo features categóricas e numéricas

In [ ]:
fig, ax = plt.subplots()
datasus_df.dtypes.value_counts().plot(kind='bar', ax=ax)
ax.set_title('Número de features por tipo de dado')
ax.set_xlabel('Tipo de dado')
ax.set_ylabel('Número de features')
plt.show()

# Analisando imbalanceamento da feature alvo

In [ ]:
datasus_df.dropna(subset=['classificacaoFinal'], inplace=True)

In [ ]:
datasus_df['classificacaoFinal'].value_counts()

# Lidando com a coluna "Sintomas"

In [ ]:
datasus_df['listaSintomas'] = datasus_df['sintomas'].str.split(',')
datasus_df.dropna(subset=['listaSintomas'], inplace=True)
datasus_df['listaSintomas'] = datasus_df['listaSintomas'].apply(lambda x: [s.lower().strip() for s in x])
datasus_df['idPaciente'] = datasus_df.index
sintomas_df = datasus_df.explode('listaSintomas')
encoded_df = pd.get_dummies(sintomas_df['listaSintomas'])
agrupado_df = sintomas_df[['idPaciente']].join(encoded_df).groupby('idPaciente').max()
resultado_df = pd.merge(datasus_df.drop(columns=['listaSintomas']), agrupado_df, left_index=True, right_index=True)
resultado_df.drop(columns=['', 'sintomas'], inplace=True)

# Lidando com a coluna "Outros Sintomas"
A coluna "outrosSintomas" necessita do mesmo tratamento que a coluna "sintomas", mas para propósitos de economia computacional, vamos dropar ela por hora

In [ ]:
resultado_df.drop(columns=['outrosSintomas', 'outrasCondicoes'], inplace=True)

In [ ]:
resultado_df.nunique().sort_values(ascending=False)

# Dropando colunas insignificantes

## Colunas de data

In [ ]:
resultado_df.drop(columns=resultado_df.filter(like='data').columns, inplace=True)
resultado_df.info()

## Coluna "source_id"

In [ ]:
resultado_df.drop(columns=['source_id'], inplace=True)

## Colunas de códigos
Muitas das colunas que possuem "código" no nome estão como tipo float64, quando na verdade são códigos únicos e categóricos

In [ ]:
resultado_df[resultado_df.select_dtypes(include=['float64']).columns].info()

In [ ]:
resultado_df[resultado_df.filter(like='codigo').columns] = resultado_df.filter(like='codigo').astype('category')
resultado_df[resultado_df.filter(like='codigo').columns].info()

In [ ]:
resultado_df[resultado_df.filter(like='codigo').columns].nunique().sort_values(ascending=False)

# Tratando colunas categóricas do tipo "object"

In [ ]:
resultado_df.select_dtypes(include=['object']).columns

In [ ]:
resultado_df.select_dtypes(include=['object']).nunique().sort_values(ascending=False)

In [ ]:
resultado_df.drop(columns=['estadoNotificacao', 'estadoNotificacaoIBGE', 'origem', 'cbo'], inplace=True)

# Tratando a coluna condições
A coluna "condições", assim como a coluna "sintomas", possui strings que determinam as condições, e portanto cada condição listada deve receber sua própria coluna booleana. No entanto, fazer esse tipo de tratamento nos dados implica numa perda de 84000 entries, então o código abaixo foi comentado para ficar como um tratamento opcional para propósitos de treinamento.

In [ ]:
resultado_df['condicoes'] = resultado_df['condicoes'].fillna('nenhuma condição')
resultado_df['listaCondicoes'] = datasus_df['condicoes'].str.split(',')
resultado_df.dropna(subset=['listaCondicoes'], inplace=True)
resultado_df['listaCondicoes'] = resultado_df['listaCondicoes'].apply(lambda x: [s.lower().strip() for s in x])
condicoes_df = resultado_df.explode('listaCondicoes')
encoded_df = pd.DataFrame()
encoded_df = pd.get_dummies(condicoes_df['listaCondicoes'])
agrupado_df = pd.DataFrame()
agrupado_df = condicoes_df[['idPaciente']].join(encoded_df).groupby('idPaciente').max()
resultado_df = pd.merge(resultado_df.drop(columns=['listaCondicoes']), agrupado_df, left_index=True, right_index=True)
resultado_df.drop(columns=['condicoes'], inplace=True)
resultado_df.info()

In [ ]:
resultado_df.drop(columns=['4 ou 5)'], inplace=True)

In [ ]:
resultado_df.drop(columns=resultado_df.columns[resultado_df.isnull().mean() > 0.9], inplace=True)

In [ ]:
resultado_df.info()

In [ ]:
for col in resultado_df.columns:
    if resultado_df[col].dtype == 'categorical':
        unique_types = resultado_df[col].map(type).unique()
        print(f"Column '{col}' has the following types: {unique_types}")

In [ ]:
resultado_df['municipioIBGE'] = resultado_df['municipioIBGE'].fillna(resultado_df['municipioIBGE'].mode()[0])
resultado_df['municipioIBGE'] = resultado_df['municipioIBGE'].apply(lambda x: resultado_df['municipioIBGE'].mode()[0] if x == 'BA' else x)
resultado_df['municipioIBGE'] = resultado_df['municipioIBGE'].apply(lambda x: int(x) if isinstance(x, float) else x)
resultado_df['municipioIBGE'] = resultado_df['municipioIBGE'].apply(lambda x: str(x) if isinstance(x, int) else x)
resultado_df['municipioIBGE'] = resultado_df['municipioIBGE'].astype('category')
resultado_df['evolucaoCaso'] = resultado_df['evolucaoCaso'].fillna('Nenhum')
resultado_df['outroBuscaAtivaAssintomatico'] = resultado_df['outroBuscaAtivaAssintomatico'].fillna('Nenhum')
resultado_df['lotePrimeiraDose'] = resultado_df['lotePrimeiraDose'].fillna('Nenhum')
resultado_df['loteSegundaDose'] = resultado_df['loteSegundaDose'].fillna('Nenhum')
resultado_df['estadoIBGE'] = resultado_df['estadoIBGE'].fillna(resultado_df['estadoIBGE'].mode()[0])
resultado_df['profissionalSeguranca'] = resultado_df['profissionalSeguranca'].fillna(resultado_df['profissionalSeguranca'].mode()[0])
resultado_df['racaCor'] = resultado_df['racaCor'].fillna(resultado_df['racaCor'].mode()[0])
resultado_df['totalTestesRealizados'] = resultado_df['totalTestesRealizados'].apply(lambda x: int(x) if isinstance(x, str) else x)

for column in resultado_df.filter(like='codigo').columns:
    resultado_df[column] = resultado_df[column].fillna(resultado_df[column].mode()[0])

resultado_df['idade'] = resultado_df['idade'].fillna(resultado_df['idade'].mean())
resultado_df['municipioNotificacaoIBGE'] = resultado_df['municipioNotificacaoIBGE'].fillna(resultado_df['municipioIBGE'])

In [ ]:
def converter_para_coluna_bool(df, col):
    df[col] = df[col].apply(lambda x: x.capitalize() if isinstance(x, str) else x)
    df[col] = df[col].map({'False': False, 'True': True})
    df[col] = df[col].astype('bool')

In [ ]:
converter_para_coluna_bool(resultado_df, 'excluido')
converter_para_coluna_bool(resultado_df, 'validado')

In [ ]:
print(resultado_df['totalTestesRealizados'].map(type).value_counts())
print(resultado_df['totalTestesRealizados'].sample(5))

In [ ]:
for column in resultado_df.select_dtypes(include=['object']).columns:
    resultado_df[column] = resultado_df[column].astype('category')

# Solucionando a falta de balanceamento no target "classificacaoFinal"


In [ ]:
# Plot a graph of the balance of the target variable "classificacaoFinal" using plotly-express
fig = px.histogram(resultado_df, x='classificacaoFinal', title='Distribuição das classes na variável alvo')
fig.show()


In [ ]:
label_encoder = LabelEncoder()
resultado_df['classificacaoFinal'] = label_encoder.fit_transform(resultado_df['classificacaoFinal'])

for column in resultado_df.drop(columns=['classificacaoFinal']).select_dtypes(include=['category']).columns:
    encoded_df = pd.get_dummies(resultado_df[column], prefix=column)
    resultado_df = pd.concat([resultado_df, encoded_df], axis=1)
    resultado_df.drop(columns=[column], inplace=True)



In [ ]:
resultado_df.to_csv(r'./datasus/datasus_preprocessed.csv', index=False)